In [2]:
import time
import numpy as np
from keras.layers import (Concatenate, Conv3D, Dropout, Input,
                          MaxPooling3D, UpSampling3D)
from keras.models import Model
from keras.optimizers import Adam
import random
import common_functions as cf

In [3]:
################# define functions
def Im2Data(Im,label,sub_size):
    L = Im.shape[0]//sub_size[0] * Im.shape[1]//sub_size[1] * Im.shape[2]//sub_size[2]
    x_val = np.zeros([L,sub_size[0],sub_size[1],sub_size[2],1])
    y_val = np.zeros([L,sub_size[0],sub_size[1],sub_size[2],1])
    l = 0
    for i in range(0,Im.shape[0]-sub_size[0]+1,sub_size[0]):
        for j in range(0,Im.shape[1]-sub_size[1]+1,sub_size[1]):
            for k in range(0,Im.shape[2]-sub_size[2]+1,sub_size[2]):
                x_val[l,:,:,:,0] = Im[i:i+sub_size[0],j:j+sub_size[1],k:k+sub_size[2]]
                y_val[l,:,:,:,0] = label[i:i+sub_size[0],j:j+sub_size[1],k:k+sub_size[2]]
                l = l + 1
    return x_val, y_val

In [4]:
def App3DUNetIm_test(model, Im, win_x, win_y, win_z, out_x, out_y, out_z):    
    stride_x = out_x
    stride_y = out_y
    stride_z = out_z
    output = np.zeros(Im.shape)

    for i in range(0,Im.shape[0],stride_x):
        for j in range(0,Im.shape[1],stride_y):
            for k in range(0,Im.shape[2]-stride_z,stride_z):
                sub = Im[i:i+stride_x,j:j+stride_y,k:k+stride_z]
                sub = np.expand_dims(np.expand_dims(sub, axis=0), axis=4)
                SM = output[i:i+out_x,j:j+out_y,k:k+out_z]
                temp_IM = model.predict(sub)
                temp_IM = np.squeeze(temp_IM)
                output[i:i+out_x,j:j+out_y,k:k+out_z] = np.maximum(SM,temp_IM)
    # dealing with boundary
    for i in range(0,Im.shape[0],stride_x):
        for j in range(0,Im.shape[1],stride_y):
            k = Im.shape[2]-stride_z
            sub = Im[i:i+stride_x,j:j+stride_y,k:k+stride_z]
            sub = np.expand_dims(np.expand_dims(sub, axis=0), axis=4)
            SM = output[i:i+out_x,j:j+out_y,k:k+out_z]
            temp_IM = model.predict(sub)
            temp_IM = np.squeeze(temp_IM)
            output[i:i+out_x,j:j+out_y,k:k+out_z] = np.maximum(SM,temp_IM)
    return output 

In [5]:
################### parameters
test_list = 0
win_x = 32 # x size of input and output, must be a power of 2
win_y = 32 # y size of input and output, must be a power of 2
win_z = 8 # z size of input and output, positive integer
learning_rate = 1
keep_p = 0.8 # 1 - drop out probability
batch_size = 5 # batch size
max_train_steps = int(2e1) # maximum number of training steps
plotting_step = int(1e1) # update of the loss plot 
sample_stride = [win_x,win_y,win_z]

In [12]:
################### paths
data_name = 'L1'
data_dir = '../../data/'+data_name+'/'

valid_IM_name = 'image_4.tif'
valid_label_name = 'label_4.tif'

model_dir = '../../data/training_result/'
folder_name = '3DUNet_'+data_name+'_test20190111'
cf.createFolder(model_dir + folder_name)

In [13]:
################### 

pad_x = int(win_x/2)
pad_y = int(win_y/2)
pad_z = int(win_z/2)
out_x = win_x
out_y = win_y
out_z = win_z

#Save variables as numpy .npz
np.savez(model_dir+folder_name+'/var.npz', win_x = win_x, win_y=win_y, win_z=win_z, learning_rate=learning_rate, batch_size = batch_size, keep_p = keep_p, plotting_step = plotting_step)


In [14]:
###################
N_images = {'L1':6, 'OP':9}

Index_matrix, train_IM_list, train_label_list = cf.get_ind_IMlist(data_dir,N_images[data_name],out_x,out_y,out_z,pad_x,pad_y,pad_z)
#Index_matrix = Index_matrix[np.logical_and(Index_matrix[:,1] % sample_stride[0] ==0,Index_matrix[:,2] % sample_stride[1] ==0,Index_matrix[:,3] % sample_stride[2] ==0),:]
#for i in range(len(train_label_list)):
#    train_label_list[i] = 1*(train_label_list[i]==1)


In [22]:
# Define model 
conv_properties = {'activation': 'relu', 'padding': 'same', 'kernel_initializer': 'he_normal'}

input_im = Input(shape=(win_x, win_y, win_z, 1), name='input_im')
#              #filters (filter size)
conv1 = Conv3D(8, (3, 3, 3), **conv_properties)(input_im)
pool1 = MaxPooling3D(pool_size=(2, 2, 1))(conv1)

conv2 = Conv3D(8, (3, 3, 3), **conv_properties)(pool1)
pool2 = MaxPooling3D(pool_size=(2, 2, 1))(conv2)

conv3 = Conv3D(4, (3, 3, 1), **conv_properties)(pool2)
pool3 = MaxPooling3D(pool_size=(2, 2, 1))(conv3)

conv4 = Conv3D(4, (3, 3, 1), **conv_properties)(pool3)
drop4 = Dropout(0.2)(conv4) # Drop out to prevent overfitting
pool4 = MaxPooling3D(pool_size=(2, 2, 1))(drop4)

conv5 = Conv3D(4, (3, 3, 1), **conv_properties)(pool4)
drop5 = Dropout(0.2)(conv5)

up6 = UpSampling3D(size=(2, 2, 1))(drop5) # handling imbalance data
up6 = Conv3D(4, (2, 2, 1), **conv_properties)(up6)
cat6 = Concatenate(axis=-1)([drop4, up6])
conv6 = Conv3D(4, (3, 3, 1), **conv_properties)(cat6)

up7 = UpSampling3D(size=(2, 2, 1))(conv6)
up7 = Conv3D(4, (2, 2, 1), **conv_properties)(up7)
cat7 = Concatenate(axis=-1)([conv3, up7])
conv7 = Conv3D(4, (3, 3, 1), **conv_properties)(cat7)

up8 = UpSampling3D(size=(2, 2, 1))(conv7)
up8 = Conv3D(8, (2, 2, 1), **conv_properties)(up8)
cat8 = Concatenate(axis=-1)([conv2, up8])  
conv8 = Conv3D(8, 3, **conv_properties)(cat8)

up9 = UpSampling3D(size=(2, 2, 1))(conv8)
up9 = Conv3D(8, (2, 2, 1), **conv_properties)(up9)
cat9 = Concatenate(axis=-1)([conv1, up9])
conv9 = Conv3D(8, (3, 3, 3), **conv_properties)(cat9)

output_im = Conv3D(1, (1, 1, 1), activation='sigmoid', name='output_im')(conv9)

model = Model(inputs=[input_im], outputs=[output_im])

model.compile(optimizer=Adam(), loss= 'binary_crossentropy', metrics=['accuracy'])

In [23]:
################### generating validation data
valid_IM, valid_label = cf.get_validation_set(data_dir,valid_IM_name,valid_label_name)

x_val, y_val = Im2Data(valid_IM,valid_label,[win_x,win_y,win_z])

In [24]:
################### training 
start_time = time.time()            

pixel_ind = np.random.randint(Index_matrix.shape[0], size = 2*max_train_steps*batch_size)
dummy_train = np.zeros([batch_size, win_x, win_y, win_z, 1])
dummy_label = np.zeros([batch_size, out_x, out_y, out_z, 1])

z = 0
s = 0
while s < max_train_steps:
    bi = 0  
    while bi < batch_size:
        m = pixel_ind[z]
        ind, i, j, k = Index_matrix[m,:]
        if ind != test_list:
            train_IM = train_IM_list[ind]
            train_label = train_label_list[ind]
            temp_train = train_IM[i:i+win_x,j:j+win_y,k:k+win_z]
            temp_label = train_label[i:i+win_x,j:j+win_y,k:k+win_z]
            ind_trans = random.randrange(8)
            if ind_trans > 3:
                temp_train = np.flip(temp_train, 0)
                temp_label = np.flip(temp_label, 0)
            temp_train = np.rot90(temp_train, ind_trans % 4 )
            temp_label = np.rot90(temp_label, ind_trans % 4 )
            dummy_train[bi,:,:,:,0] = temp_train
            dummy_label[bi,:,:,:,0] = temp_label
            bi = bi + 1     
        z = z + 1

    history = model.fit(x=dummy_train, y=dummy_label, batch_size=batch_size, epochs=1, verbose=0, callbacks = None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
    s = s + 1
  
    if s % plotting_step == 0:
        model.save('../../data/training_result/'+folder_name+'/model'+'_test_on_'+data_name+'_'+str(test_list+1)+'_'+str(s))
        valid_loss_acc = model.evaluate(x=x_val,y=y_val)        
        print("--- %s seconds ---" % (time.time() - start_time))      


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv3d_14/convolution}}]]
	 [[{{node Mean_1}}]]

In [18]:
########### Save session
model.save('../../data/training_result/'+folder_name+'/model'+'_test_on_'+data_name+'_'+str(test_list+1))


In [19]:
################### loading test data
test_IM_name = 'image_'+str(test_list+1)+'.tif' 
test_label_name = 'label_'+str(test_list+1)+'.tif' 
test_IM, test_label = cf.get_data(data_dir+test_IM_name,data_dir+test_label_name)
#test_label = 1*(test_label==1)

In [20]:
####################### testing 
test_output = App3DUNetIm_test(model, test_IM, win_x, win_y, win_z, out_x, out_y, out_z)


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv3d_1/convolution}}]]
	 [[{{node output_im/Sigmoid}}]]

In [21]:
####################### for internal use only
import scipy.io as sio
sio.savemat('../../data/training_result/'+folder_name+'/output'+str(test_list+1)+'.mat',{'IM':np.uint8(test_output*255),'Original':np.uint8(test_IM*255),'label':np.uint8(test_label*255)})

np.savez(model_dir+folder_name+'/var.npz', win_x = win_x, win_y=win_y, win_z=win_z, learning_rate=learning_rate, batch_size = batch_size, keep_p = keep_p, plotting_step = plotting_step, z = z, s = s)


NameError: name 'test_output' is not defined